In [7]:
import sys, os
sys.path.append(os.path.abspath('../backend/app'))

from adzuna_api import search_adzuna_jobs
from job_ad_api import match_jobs_by_skills


In [8]:
# 📦 Imports
import threading, time
from tqdm.auto import tqdm
import sys, os
sys.path.append(os.path.abspath('../backend/app'))

from adzuna_api import search_adzuna_jobs
from job_ad_api import match_jobs_by_skills
import pandas as pd


/Users/kunalgurung/TechSkillSync/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import threading
threading.enumerate()


[<_MainThread(MainThread, started 8463847232)>,
 <Thread(IOPub, started daemon 6141751296)>,
 <Heartbeat(Heartbeat, started daemon 6158577664)>,
 <Thread(Thread-1, started daemon 6176550912)>,
 <Thread(Thread-2, started daemon 6193377280)>,
 <ControlThread(Control, started daemon 6210203648)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 6227030016)>,
 <TMonitor(Thread-5, started daemon 6244429824)>]

In [10]:
keyword = input("Enter job keyword (e.g., data analyst): ")
location = input("Enter job location (e.g., Sydney): ")
skills_input = input("Enter your skills (comma-separated, e.g., Python,SQL,Excel): ")
user_skills = [skill.strip() for skill in skills_input.split(",") if skill.strip()]


In [11]:
# Spinner animation
def loading_dots(stop_event):
    while not stop_event.is_set():
        for dots in [".  ", ".. ", "..."]:
            print(f"\r🔄 Fetching jobs{dots}", end="", flush=True)
            time.sleep(0.5)

stop_event = threading.Event()
spinner_thread = threading.Thread(target=loading_dots, args=(stop_event,))
spinner_thread.start()

# Actual fetch
jobs = search_adzuna_jobs(keyword, location)

stop_event.set()
spinner_thread.join()
print(f"\r✅ Done fetching {len(jobs)} jobs!     ")


✅ Done fetching 10 jobs!     


In [12]:
# Custom version of match_jobs_by_skills with progress bar
def match_with_progress(jobs, skills):
    matched_jobs = []
    for job in tqdm(jobs, desc="🔍 Matching skills"):
        desc = job.get("description", "")
        matched = [s for s in skills if s.lower() in desc.lower()]
        score = len(matched)
        matched_jobs.append({
            **job,
            "matched_skills": matched,
            "match_score": score
        })
    matched_jobs.sort(key=lambda x: x["match_score"], reverse=True)
    return matched_jobs

# Match and show
matched = match_with_progress(jobs, user_skills)


🔍 Matching skills: 100%|██████████| 10/10 [00:00<00:00, 64133.09it/s]


In [13]:
df = pd.DataFrame(matched)
df[["title", "company", "location", "match_score", "matched_skills", "url"]]


,title,company,location,match_score,matched_skills,url
0,Data Analyst,Pyramid Global Technologies,"The Rocks, Sydney",1,[sql],https://www.adzuna.com.au/land/ad/5114848179?s...
1,Data Analyst,Luxury Escapes,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5139802599?s...
2,Data Analyst,Investa,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5111306962?s...
3,Data Analyst,OMD Australia,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5134493751?s...
4,Data Analyst,Luxury Escapes,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5114871500?s...
5,Data Analyst,Westpac,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5138474598?s...
6,Data Analyst,HAYS,"Sydney CBD, Sydney",0,[],https://www.adzuna.com.au/land/ad/5141706818?s...
7,Data Analyst,Ubank,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5140041628?s...
8,Data Analyst,Endeavour Group,"Surry Hills, Sydney",0,[],https://www.adzuna.com.au/land/ad/5135186168?s...
9,Data Analyst,Macquarie,"The Rocks, Sydney",0,[],https://www.adzuna.com.au/land/ad/5133176420?s...
